# Kerchunk in Practice

In this notebook, we demonstrate how to create a kerchunk reference file for one and then multiple publicly available NetCDF files and how to open a kerchunk store with `xarray`.

## How to create a Kerchunk Store

We can use the publicly available [NEX GDDP CMIP6](https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-gddp-cmip6) dataset for this exercise. This dataset is provided by NASA and publicly available on AWS S3. You can browse that data in the [nex-gddp-cmip6 file browser](https://nex-gddp-cmip6.s3.us-west-2.amazonaws.com/index.html#NEX-GDDP-CMIP6/).

In [1]:
import fsspec
import json
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr
import os
from tempfile import TemporaryDirectory
import ujson
import xarray as xr

In [2]:
# Since there are a number of CMIP6 models and variables to chose from, we make those selections variables
model = "ACCESS-CM2"
variable = "tasmax"

# Note we are only reading historical data here, but model data is available for SSPs (Shared Socio-economic Pathways) as well.
s3_path = f"s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/{model}/historical/r1i1p1*/{variable}/*"

# Initiate fsspec filesystem for reading
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=False)

# Retrieve list of available data
file_paths = fs_read.glob(s3_path)
print(f"{len(file_paths)} discovered from {s3_path}")

65 discovered from s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1*/tasmax/*


To start, we are just going to create a single reference file for a single NetCDF file.

In [3]:
netcdf_file = file_paths[0]
netcdf_file

'nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc'

In [4]:
# Define a function to generate the kerchunk file so we can use it again for other files
def generate_json_reference(input_file, temp_dir: str):
    """
    Use Kerchunk's `SingleHdf5ToZarr` method to create a `Kerchunk` index from a NetCDF file.
    """
    source_mode = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")
    with fs_read.open(input_file, **source_mode) as infile:
        print(f"Running kerchunk generation for {input_file}...")
        h5chunks = SingleHdf5ToZarr(infile, input_file, inline_threshold=300)
        fname = input_file.split("/")[-1].strip(".nc")
        outf = os.path.join(temp_dir, f"{fname}.json")
        with open(outf, "wb") as f:
            f.write(ujson.dumps(h5chunks.translate()).encode())
        return outf

In [5]:
# Create a temporary directory to store the .json reference files.
# Alternately, you could write these to cloud storage.
td = TemporaryDirectory()
temp_dir = td.name
print(f"Writing single file references to {temp_dir}")

Writing single file references to /var/folders/jh/_03qbqf130l8hjh8rpc6f4_c0000gn/T/tmp1076qeep


In [6]:
single_kerchunk_reference = generate_json_reference(netcdf_file, temp_dir)

Running kerchunk generation for nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc...


In [7]:
import json
from pprint import pprint

# Read and load the JSON file
with open(single_kerchunk_reference, 'r') as file:
    data = json.load(file)

# Print the first 5 items
# keys = data['refs'].slice('.zattrs', '.zgroup'
# pprint(data['refs'])
keys_to_select = ['.zgroup', '.zarray', 'lat/.zarray']

selected_data = {k: v for k, v in data['refs'].items() if k in keys_to_select}
type(selected_data)

dict

Check that it worked.

In [9]:
# open dataset as zarr object using fsspec reference file system and Xarray
fs_single = fsspec.filesystem(
    "reference", fo=single_kerchunk_reference, remote_protocol="https"
)
single_map = fs_single.get_mapper("")

In [10]:
ds_single = xr.open_dataset(single_map, engine="zarr", backend_kwargs=dict(consolidated=False))
ds_single

<xarray.Dataset>
Dimensions:  (lat: 600, lon: 1440, time: 365)
Coordinates:
  * lat      (lat) float64 0.0 1.235e-259 0.0 ... 4.673e+164 7.53e+252
  * lon      (lon) float64 0.0 6.013e-154 0.0 ... 3.636e+228 8.7e+199 5.351e-80
  * time     (time) datetime64[ns] 1950-01-01T12:00:00 ... 1950-12-31T12:00:00
Data variables:
    tasmax   (time, lat, lon) float32 ...
Attributes: (12/22)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  CSIRO-ARCCSS
    cmip6_license:         CC-BY-SA 4.0
    cmip6_source_id:       ACCESS-CM2
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    scenario:              historical
    source:                BCSD
    title:                 ACCESS-CM2, r1i1p1f1, historical, global downscale...
    tracking_id:           f85d4c2e-48e4-484f-aad4-6a3f30a04326
    variant_label:         r1i1p1f1
    version:               1.0

It worked! But we can do even better. What if you want to open multiple NetCDF files with `xarray`? Let's create kerchunk references for 3 files and then combine them.

In [12]:
subset_files = file_paths[0:3]
subset_files

['nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc',
 'nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1951.nc',
 'nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1952.nc']

In [13]:
# Iterate through filelist to generate Kerchunked files. Good use for `dask.bag`, see: https://docs.dask.org/en/stable/bag.html.
output_files = []
for single_file in subset_files:
    out_file = generate_json_reference(single_file, temp_dir)
    output_files.append(out_file)

Running kerchunk generation for nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc...
Running kerchunk generation for nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1951.nc...
Running kerchunk generation for nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1952.nc...


In [14]:
output_files

['/var/folders/jh/_03qbqf130l8hjh8rpc6f4_c0000gn/T/tmp1076qeep/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.json',
 '/var/folders/jh/_03qbqf130l8hjh8rpc6f4_c0000gn/T/tmp1076qeep/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1951.json',
 '/var/folders/jh/_03qbqf130l8hjh8rpc6f4_c0000gn/T/tmp1076qeep/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1952.json']

In [15]:
# combine individual references into single consolidated reference
mzz = MultiZarrToZarr(
    output_files,
    remote_protocol='s3',
    remote_options={'anon': anon},
    concat_dims=['time'],
    coo_map={"time": "cf:time"},
    inline_threshold=0
)
multi_kerchunk = mzz.translate()

NameError: name 'anon' is not defined

In [ ]:
# Write kerchunk .json record
output_fname = os.path.join(temp_dir, f"combined_CMIP6_{temporal_resolution}_{model}_{variable}_kerchunk.json")
with open(f"{output_fname}", "wb") as f:
    print(f"Writing combined kerchunk reference file {output_fname}")
    f.write(ujson.dumps(multi_kerchunk).encode())

In [ ]:
# open dataset as zarr object using fsspec reference file system and Xarray
fs_multi = fsspec.filesystem(
    "reference",
    fo=multi_kerchunk,
    remote_protocol="s3"
)
multi_map = fs_multi.get_mapper("")

In [ ]:
ds_multi = xr.open_dataset(multi_map, engine="zarr", backend_kwargs=dict(consolidated=False))
ds_multi

Cool! Now we have 1096 days (3 years) of data.

## How to read a Kerchunk Store

We've already demonstrated how to open the datasets with `Xarray`:

```python
fs_multi = fsspec.filesystem(
    "reference",
    fo=multi_kerchunk,
    remote_protocol="s3"
)
```

Let's take it line by line to understand what's happening.

1. `fsspec.filesystem` is used to open the kerchunk reference. It is not necessary to have kerchunk installed to read data.
2. The first argument to `fsspec.filesystem` is the protocol. In the case of a kerchunk reference the protocol is the string "reference".
3. The `fo` argument is the set of reference files used to create a `ReferenceFileSystem` instance.
4. The remote_protocol argument is he protocol of the filesystem on which the references will be evaluated(unless fs is provided). If not given, will be derived from the first URL that has a protocol in the templates or in the references.

Notice how the `fs_multi` object we've created is a `fsspec.implementations.reference.ReferenceFileSystem`.

In [ ]:
type(fs_multi)

Read about all the options for a [`fsspec.ReferenceFileSystem` in the `fsspec` docs](https://filesystem-spec.readthedocs.io/en/latest/_modules/fsspec/implementations/reference.html).

One other common situation is to load data over HTTP (as opposed to a local filesystem or via the S3 protocol). Here's an example from the kerchunk case studies that loads a reference file and data files over HTTP:

In [ ]:
zarr_all_url='https://sentinel-1-global-coherence-earthbigdata.s3.us-west-2.amazonaws.com/data/wrappers/zarr-all.json'

mapper = fsspec.get_mapper(
    'reference://',
    fo=zarr_all_url,
    target_protocol='http',
    remote_protocol="http"
)
dataset = xr.open_dataset(
    mapper, engine='zarr', backend_kwargs={'consolidated': False}
)
dataset

Because xarray uses fsspec to read data, you can also bypass creating a fsspec object explicitly. Here's an example using of opening a kerchunk reference generated with pangeo-forge for the NOAA 1/4° daily Optimum Interpolation Sea Surface Temperature (or daily OISST) Climate Data Record (CDR).

In [ ]:
import xarray as xr
url = "https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/pangeo-forge/aws-noaa-oisst-feedstock/aws-noaa-oisst-avhrr-only.zarr/reference.json"
ds = xr.open_dataset(
    "reference://",
    engine='zarr',
    backend_kwargs={
        'consolidated': False,
        'storage_options': {
            'fo': url,
            'remote_options': {'anon': True},
            'remote_protocol': 's3'}},
    chunks={})
ds

## Other Examples of Existing Kerchunk Data

* [Case Studies on Kerchunk Docs page](https://fsspec.github.io/kerchunk/cases.html)

# Additional resources

* [Fake it until you make it — Using Kerchunk to read NetCDF4 data on AWS S3 as Zarr for rapid data access by Lucas Sterzinger](https://medium.com/pangeo/fake-it-until-you-make-it-reading-goes-netcdf4-data-on-aws-s3-as-zarr-for-rapid-data-access-61e33f8fe685)